In [18]:
import pandas as pd
import numpy as np

df = pd.read_excel("bases/relatorio_ra-todas_05-12-2025.xlsx", engine="calamine")
df_lista = pd.read_excel("listagem/listagem.xlsx", engine="calamine")

In [19]:
df.columns  

Index(['Loja', 'Cod. Loja', 'CNPJ', 'CLISAP', 'Grife', 'Artigo', 'Cor',
       'Tamanho', 'Sexo', 'Grupo', 'Linha', 'Vel. de Vendas', 'Lead Time',
       'Cobertura', 'Mínimo', 'Alvo', 'Em Loja', 'Em Transito', 'Em Carteira',
       'Necessidade', 'Ruptura', 'Ressuprir', 'Cancelada', 'Faltou no Estoque',
       'Atendido (SAP)', 'Valor', 'Nr. Pedido', 'Data Pedido', 'Data Embarque',
       'Data de Geração', 'Status Item', 'Status Pedido', 'Retorno Pedido'],
      dtype='object')

In [20]:
df['Retorno Pedido'] = df['Retorno Pedido'].str.split(',').str[0]

In [21]:
df["Data de Geração"].value_counts()

Data de Geração
2025-12-05    61808
Name: count, dtype: int64

In [24]:
df = df.merge(df_lista[['Cod. Loja', 'Franqueado']], how='left', on='Cod. Loja')

In [25]:
df.isnull().sum()

Loja                    0
Cod. Loja               0
CNPJ                    0
CLISAP                  0
Grife                   0
Artigo                  0
Cor                     0
Tamanho                 0
Sexo                    0
Grupo                   0
Linha                   0
Vel. de Vendas          0
Lead Time               0
Cobertura               0
Mínimo                  0
Alvo                    0
Em Loja                 0
Em Transito             0
Em Carteira             0
Necessidade             0
Ruptura                 0
Ressuprir               0
Cancelada               0
Faltou no Estoque       0
Atendido (SAP)          0
Valor                6146
Nr. Pedido           6146
Data Pedido             0
Data Embarque           0
Data de Geração         0
Status Item             0
Status Pedido           0
Retorno Pedido          0
Franqueado              0
dtype: int64

In [26]:
filtro_cencelada = df['Cancelada'] > 0

TD_cancelado = df[filtro_cencelada]


TD_cancelado = TD_cancelado.groupby('Retorno Pedido').agg(
    total_registros=('Retorno Pedido', 'size'),
    total_cancelados=('Cancelada', 'sum'),
    total_lojas=('Loja', 'nunique'),
    total_franqueados=('Franqueado', 'nunique')

).reset_index()



In [27]:
total_geral = TD_cancelado['total_registros'].sum()
TD_cancelado['%proporção'] = TD_cancelado['total_registros'] / total_geral

linha_total = {
    'Retorno Pedido': 'TOTAL',
    'total_registros': TD_cancelado['total_registros'].sum(),
    'total_cancelados': TD_cancelado['total_cancelados'].sum(),
    'total_lojas': TD_cancelado['total_lojas'].sum(),
    'total_franqueados': TD_cancelado['total_franqueados'].sum(),
    '%proporção': 1.0


}

df_total = pd.DataFrame([linha_total])
td_final = pd.concat([TD_cancelado, df_total], ignore_index=True)

def fmt_milhar(valor):
    return f"{valor:,.0f}".replace(',', ".")

def fmt_pct(valor):
    return f"{valor:.0%}"

colunas_numericas = ['total_registros', 'total_cancelados', 'total_lojas', 'total_franqueados']

for col in colunas_numericas:
    td_final[col] = td_final[col].apply(fmt_milhar)


td_final['%proporção'] = td_final['%proporção'].apply(fmt_pct)

td_final.head()

,Retorno Pedido,total_registros,total_cancelados,total_lojas,total_franqueados,%proporção
0,Ord./fornec.não é possível,787,3.934,6,5,13%
1,SUSPENSO COBRANÇA,5.359,19.705,43,15,87%
2,TOTAL,6.146,23.639,49,20,100%
